In [1]:
%pip install feature_engine

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for feature_engine from https://files.pythonhosted.org/packages/62/60/77fcc9d3cfaabab34027aa8ea0025c5e2d4cf9561fa9725a38f0785b43aa/feature_engine-1.6.2-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/328.9 kB ? eta -:--:--
   --- ------------------------------------ 30.7/328.9 kB 1.3 MB/s eta 0:00:01
   -------------- ------------------------- 122.9/328.9 kB 1.4 MB/s eta 0:00:01
   ---------------------------------- ----- 286.7/328.9 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 328.9/328.9 kB 2.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [189]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from feature_engine.transformation import LogTransformer
from feature_engine.timeseries.forecasting import LagFeatures, WindowFeatures
from sklearn.pipeline import make_pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [222]:
df = pd.read_csv('train.csv')

In [223]:
df.head()

,merchant_id,month_id,merchant_source_name,settlement_period,working_type,mcc_id,merchant_segment,net_payment_count
0,merchant_43992,202307,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_197,Segment - 2,15106
1,merchant_43992,202301,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_197,Segment - 2,16918
2,merchant_43992,202305,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_197,Segment - 2,13452
3,merchant_43992,202308,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_197,Segment - 2,16787
4,merchant_43992,202302,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_197,Segment - 2,12428


In [224]:
df['month'] = df['month_id'].apply(lambda x: x % 100)
df['year'] = df['month_id'] // 100

In [225]:
df.tail(100)

,merchant_id,month_id,merchant_source_name,settlement_period,working_type,mcc_id,merchant_segment,net_payment_count,month,year
291042,merchant_46399,202309,Merchant Source - 1,Settlement Period - 1,Working Type - 6,mcc_42,Segment - 4,3,9,2023
291043,merchant_45656,202309,Merchant Source - 2,Settlement Period - 1,Working Type - 6,mcc_73,Segment - 4,3,9,2023
291044,merchant_40359,202309,Merchant Source - 2,Settlement Period - 1,Working Type - 5,mcc_116,Segment - 4,3,9,2023
291045,merchant_41644,202309,Merchant Source - 1,Settlement Period - 1,Working Type - 6,mcc_73,Segment - 4,4,9,2023
291046,merchant_23781,202309,Merchant Source - 1,Settlement Period - 1,Working Type - 5,mcc_42,Segment - 4,4,9,2023
...,...,...,...,...,...,...,...,...,...,...
291137,merchant_67083,202309,Merchant Source - 1,Settlement Period - 1,Working Type - 6,mcc_169,Segment - 4,4,9,2023
291138,merchant_4263,202309,Merchant Source - 2,Settlement Period - 1,Working Type - 6,mcc_73,Segment - 4,4,9,2023
291139,merchant_11346,202309,Merchant Source - 1,Settlement Period - 1,Working Type - 6,mcc_169,Segment - 4,3,9,2023
291140,merchant_21397,202309,Merchant Source - 1,Settlement Period - 1,Working Type - 6,mcc_168,Segment - 4,3,9,2023


In [226]:
df.drop(columns = df.columns[1: -3], inplace = True)

In [227]:
df.head()

,merchant_id,net_payment_count,month,year
0,merchant_43992,15106,7,2023
1,merchant_43992,16918,1,2023
2,merchant_43992,13452,5,2023
3,merchant_43992,16787,8,2023
4,merchant_43992,12428,2,2023


In [228]:
df = df.sort_values('merchant_id')

In [229]:
df.head()

,merchant_id,net_payment_count,month,year
234371,merchant_1,3,10,2022
234370,merchant_1,3,8,2021
249226,merchant_10,24,12,2021
249227,merchant_10,9,3,2022
249225,merchant_10,7,2,2022


In [230]:
standard = pd.DataFrame({
    'year': [2020] * 12 + [2021] * 12 + [2022] * 12 + [2023] * 9,
    'month': list(range(1, 13)) * 3 + list(range(1, 10)),
    'months_elabsed': np.arange(45) + 1,
})

In [231]:
standard.head(15)

,year,month,months_elabsed
0,2020,1,1
1,2020,2,2
2,2020,3,3
3,2020,4,4
4,2020,5,5
5,2020,6,6
6,2020,7,7
7,2020,8,8
8,2020,9,9
9,2020,10,10


In [232]:
test_data = pd.DataFrame({
    'year' : 2023,
    'month': [10, 11, 12],
    'months_elabsed': [46, 47, 48],
    'spring': False,
    'summer': False,
    'fall': [True, True, False],
    'winter': [False, False, True],
    'net_payment_count': 0,
})

In [233]:
results = pd.DataFrame(columns = ['id', 'net_payment_count'])

In [234]:
log = LogTransformer()

In [235]:
df.head()

,merchant_id,net_payment_count,month,year
234371,merchant_1,3,10,2022
234370,merchant_1,3,8,2021
249226,merchant_10,24,12,2021
249227,merchant_10,9,3,2022
249225,merchant_10,7,2,2022


In [236]:
df= df[df["merchant_id"]=="merchant_976"]

In [237]:
df["net_payment_count"]

246972    3
246971    4
246970    3
246968    3
246969    4
Name: net_payment_count, dtype: int64

In [2]:
y = results
y_log = results
model = XGBRegressor()

a = df
X_train = []
for index, merchant in enumerate(df.groupby('merchant_id')):

    sub = pd.merge(standard, merchant[1].drop(columns = 'merchant_id'), 'outer', ['year', 'month']).fillna(0)
    merchant_id = merchant[0]
    
    if sub['net_payment_count'].iloc[-1] <= 0:
        for i in range(3):
            results.loc[results.index.size] = dict(
                id = f'{202310 + i}{merchant_id}', 
                net_payment_count = 0
            )
            
    else:
        test_data.index = [45, 46, 47]

        sub['spring'] = (sub['month'] == 3) | (sub['month'] == 4) | (sub['month'] == 5)
        sub['summer'] = (sub['month'] >= 6) & (sub['month'] <= 8)
        sub['fall'] = sub['month'].apply(lambda x: x in [9, 10, 11]) 
        sub['winter'] = sub['month'].isin([12, 1, 2])
        ###############################################################################################
        sub = make_pipeline(
            LagFeatures('net_payment_count', [4, 5, 6]),
            WindowFeatures('net_payment_count', 5, 5, 'mean', 5),
            WindowFeatures('net_payment_count', 3, functions = 'sum', periods = 5),
        ).fit_transform(pd.concat([sub, test_data]))

        train = sub.iloc[:-3]
        test = sub.iloc[-3:]

        X_train = train.drop(columns = ['net_payment_count'])
        X_test = test.drop(columns = ['net_payment_count'])
        print(X_train.head(45))

        try:
            y_train = log.fit_transform((train['net_payment_count'].values.reshape(-1, 1)) + 1)
            model = XGBRegressor()
            y_pred = model.fit(X_train, y_train).predict(X_test).reshape(-1, 1)
            y_pred -= 1
            y_pred = np.round(log.inverse_transform([y_pred])).values.flatten()

        except ValueError:
            y_train = train['net_payment_count']
            model = XGBRegressor()
            y_pred = np.round(model.fit(X_train, y_train).predict(X_test))
        y = train['net_payment_count']
        y_log = y_train
        for i in range(3):
            results.loc[results.shape[0]] = {
                'id': f'20231{i}{merchant_id}',
                'net_payment_count': y_pred[i],
            }
        print(results.head())
        a = sub
        
        break

NameError: name 'results' is not defined

def plot_importance(model, features, num=len(X_train), save=False):
    feature_imp = pd.DataFrame({'Value': model.feature_importances_, 'Feature': features.columns})
    plt.figure(figsize=(10, 10))
    sns.set(font_scale=1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                                     ascending=False)[0:num])
    plt.title('Features')
    plt.tight_layout()
    plt.show()
    if save:
        plt.savefig('importances.png')


plot_importance(model, X_train, num=10)

In [239]:
y

,id,net_payment_count
0,202310merchant_976,0
1,202311merchant_976,0
2,202312merchant_976,0


In [240]:
sub["net_payment_count"]

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    4.0
22    0.0
23    4.0
24    0.0
25    3.0
26    0.0
27    0.0
28    3.0
29    0.0
30    0.0
31    0.0
32    3.0
33    0.0
34    0.0
35    0.0
36    0.0
37    0.0
38    0.0
39    0.0
40    0.0
41    0.0
42    0.0
43    0.0
44    0.0
Name: net_payment_count, dtype: float64

In [241]:
# results.to_csv('submission.csv', index = False)

In [243]:
 
import numpy as np
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(x =sub.index , y = sub["net_payment_count"], mode='lines+markers'))
fig.update_layout(title="merchant_976", title_x=0.5)
fig.update_traces(marker_color='black', line_color='blue')
fig.show()